### Objectives

First, create a graph representation of the information we have. 
This would be information about the elements, and the nodes. 
Using this, we will feed it into our SVM, or a Random Forest, and see from there onwards

In [115]:
import pandas as pd
import igraph as ig

def Mitograph_Graph(FileName):
    # Read information from the gnet file, and create a graph object
    # generate a function that takes in the filename and gives out the graph
    G = pd.read_table(FileName + '.gnet', skiprows=1, sep='\t', names=['Source', 'Target', 'Length'])
    graph = ig.Graph.TupleList(G.itertuples(index=False), directed=False, edge_attrs=['Length'])
    layout = graph.layout_auto()  # Automatic layout calculation
    graph.vs['degree'] = graph.degree()
    # ig.plot(graph, layout=layout)
    Vol = pd.read_table(FileName + '.cc', skiprows=0, sep='\t')
    Vol.columns = ['node', 'cc', 'vol_cc']

    ids = [int(node['name']) for node in graph.vs]
    graph.vs['cc_vol'] = [Vol.loc[Vol['node'] == node, 'vol_cc'].values[0] for node in ids]
    graph.vs['cc'] = [Vol.loc[Vol['node'] == node, 'cc'].values[0] for node in ids]
    return graph, Vol

In [136]:
df_dir = '/home/mitosim2/Desktop/BIRATAL/Tcdd11s_subsetted/AnalysisFiles_TCDD1nM4hr1_Subset/structural/TCDD1nM4hr1_Subset_structural'
df_dir_2 = '/home/mitosim2/Desktop/BIRATAL/Tcdd11s_subsetted/AnalysisFiles_TCDD1nM4hr1_Subset/full_Table_TCDD1nM4hr1_Subset.csv' 
df = pd.read_csv(df_dir_2)
graph, Vol = Mitograph_Graph(df_dir)

In [134]:
df = df[['node', 'cc_x', 'line_id', 'element_pixel_intensity_ratio']]


In [138]:
graph.ecount()

282

In [139]:
len(df['line_id'].unique()) 

282

In [140]:
df.drop_duplicates(subset=['line_id'], inplace=True)

In [145]:
tempdf = df[['node', 'line_id', 'degree']]
tempdf

,node,line_id,degree
0,NaN,130,NaN
7,116.0,210,1.0
11,315.0,51,1.0
34,227.0,277,3.0
46,232.0,126,1.0
...,...,...,...
2326,324.0,42,1.0
2337,327.0,38,1.0
2341,353.0,13,1.0
2371,364.0,2,1.0


In [130]:
df_node_2 = df.copy()
df_node_2.rename(columns={'node': 'node_2'}, inplace=True)
df_node_2

,node_2,cc_x,line_id,element_pixel_intensity_ratio
0,NaN,0,130,0.097645
1,NaN,0,130,0.097645
2,NaN,0,130,0.097645
3,NaN,0,130,0.097645
4,NaN,0,130,0.097645
...,...,...,...,...
2387,NaN,104,1,0.074575
2388,NaN,104,1,0.074575
2389,NaN,104,1,0.074575
2390,NaN,104,1,0.074575


In [131]:
result_df = df.merge(df_node_2, on='line_id')
result_df = result_df[result_df['node'] != result_df['node_2']]

In [132]:
result_df.drop_duplicates(subset=['line_id'])

,node,cc_x_x,line_id,element_pixel_intensity_ratio_x,node_2,cc_x_y,element_pixel_intensity_ratio_y
0,NaN,0,130,0.097645,NaN,0,0.097645
50,116.0,0,210,0.096132,NaN,0,0.096132
66,315.0,0,51,0.047447,NaN,0,0.047447
2182,227.0,0,277,0.059724,NaN,0,0.059724
2758,232.0,0,126,0.058986,NaN,0,0.058986
...,...,...,...,...,...,...,...
32149,324.0,100,42,0.050332,NaN,100,0.050332
32270,327.0,101,38,0.097924,NaN,101,0.097924
32286,353.0,102,13,0.061762,NaN,102,0.061762
33186,364.0,103,2,0.070354,NaN,103,0.070354
